In [3]:
import os
import pandas as pd
import json
import numpy as np

import cv2

In [4]:
data_path = '../data/20221010_centernet_red_cam0/KITTI/detection/training'
img_path = os.path.join(data_path,'image_2')
ann_path = os.path.join(data_path,'label_2')

In [5]:
img_list = sorted([os.path.join(img_path,x) for x in os.listdir(img_path)])
ann_list = sorted([os.path.join(ann_path,x) for x in os.listdir(ann_path)])

header = ['class','trunc','occlusion','alphax','xmin','ymin','xmax','ymax','h','w','l','x','y','z','rx','ry','rz','alphay']

In [6]:
idx = 0

img = img_list[idx]
img_name = os.path.splitext(os.path.basename(img))[0]
ann = pd.read_csv(ann_list[idx],sep = ' ',names=header)
img_name

'000000'

In [7]:
data = dict()
data['images'] = list()
data['type'] = 'instances'
data['annotations'] = list()
data['categories'] = list()

In [8]:
data['categories'] = [{"supercategory":"none","id":1,"name":"aeroplane"},{"supercategory":"none","id":2,"name":"bicycle"},{"supercategory":"none","id":3,"name":"bird"},{"supercategory":"none","id":4,"name":"boat"},{"supercategory":"none","id":5,"name":"bottle"},{"supercategory":"none","id":6,"name":"bus"},{"supercategory":"none","id":7,"name":"car"},{"supercategory":"none","id":8,"name":"cat"},{"supercategory":"none","id":9,"name":"chair"},{"supercategory":"none","id":10,"name":"cow"},{"supercategory":"none","id":11,"name":"diningtable"},{"supercategory":"none","id":12,"name":"dog"},{"supercategory":"none","id":13,"name":"horse"},{"supercategory":"none","id":14,"name":"motorbike"},{"supercategory":"none","id":15,"name":"person"},{"supercategory":"none","id":16,"name":"pottedplant"},{"supercategory":"none","id":17,"name":"sheep"},{"supercategory":"none","id":18,"name":"sofa"},{"supercategory":"none","id":19,"name":"train"},{"supercategory":"none","id":20,"name":"tvmonitor"}]

In [9]:
ann_id = 0

for idx in range(len(img_list)):
    img_p = img_list[idx]
    im_h,im_w,_ = cv2.imread(img_p).shape
    img_id = np.int64(int(os.path.splitext(os.path.basename(img_p))[0]))
    img_filename = os.path.basename(img_p)
    
    ann = pd.read_csv(ann_list[idx],sep = ' ',names=header)
    
#     images
    img_dict = dict()
    img_dict['file_name'] = img_filename
    img_dict['height'] = im_h
    img_dict['width'] = im_w
    img_dict['id'] = img_id
    
    data['images'].append(img_dict)
#     annotations
    for i in range(len(ann)):
        box  = ann.iloc[i]
        
        xmin = box['xmin']
        ymin = box['ymin']
        xmax = box['xmax']
        ymax = box['ymax']
        
        h = abs(ymax-ymin)
        w = abs(xmax-xmin)
        
        area = int(h*w)
        iscrowd = 0
        
        bbox = [xmin,ymin,xmax,ymax]
        category_id = 1
        
        ann_id = ann_id+1
        ignore = 0
        
        ann_dict = dict()
        ann_dict['segmentation'] = list()
        ann_dict['area'] = area
        ann_dict['iscrowd'] = iscrowd
        ann_dict['image_id'] = img_id
        ann_dict['bbox'] = bbox
        ann_dict['category_id'] = category_id
        ann_dict['id'] = ann_id
        ann_dict['ignore'] = ignore
        
        data['annotations'].append(ann_dict)

In [10]:
data

{'images': [{'file_name': '000000.jpg', 'height': 512, 'width': 512, 'id': 0},
  {'file_name': '000001.jpg', 'height': 512, 'width': 512, 'id': 1},
  {'file_name': '000002.jpg', 'height': 512, 'width': 512, 'id': 2},
  {'file_name': '000003.jpg', 'height': 512, 'width': 512, 'id': 3},
  {'file_name': '000004.jpg', 'height': 512, 'width': 512, 'id': 4},
  {'file_name': '000005.jpg', 'height': 512, 'width': 512, 'id': 5},
  {'file_name': '000006.jpg', 'height': 512, 'width': 512, 'id': 6},
  {'file_name': '000007.jpg', 'height': 512, 'width': 512, 'id': 7},
  {'file_name': '000008.jpg', 'height': 512, 'width': 512, 'id': 8},
  {'file_name': '000009.jpg', 'height': 512, 'width': 512, 'id': 9},
  {'file_name': '000010.jpg', 'height': 512, 'width': 512, 'id': 10},
  {'file_name': '000011.jpg', 'height': 512, 'width': 512, 'id': 11},
  {'file_name': '000012.jpg', 'height': 512, 'width': 512, 'id': 12},
  {'file_name': '000013.jpg', 'height': 512, 'width': 512, 'id': 13},
  {'file_name': '000

In [12]:
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [13]:
with open('result.json', 'w') as fp:
    json.dump(data, fp,cls=NpEncoder)

In [16]:
len(data['annotations'])

5306